In [1]:
import pandas as pd
import json
from sseclient import SSEClient as EventSource
from bloom_filter import BloomFilter

In [2]:
url = 'https://stream.wikimedia.org/v2/stream/recentchange'
wiki = 'enwiki' #Client side filter
counter = 0
changes = []
maxEvents =  5000 # print n events and stop
for event in EventSource(url):
    if event.event == 'message':
        try:
            change = json.loads(event.data)
        except ValueError:
            continue
        if change['wiki'] == wiki:
            #print(change)
            # print('{user} ====== edited  ========{title} ===={wiki}'.format(**change))
            changes.append(change)
            counter += 1
            if counter > maxEvents:
                break

In [ ]:
train = pd.json_normalize(changes)
train.to_pickle('train.pkl')
train.shape

In [3]:
train = pd.read_pickle('train.pkl')

In [4]:
bots = train.loc[train['bot']==True, 'user'].unique().tolist()

In [5]:
bloom_filter = BloomFilter(n_elements=len(bots), fp_prob=0.1)

Initiazied filter of size 186


In [6]:
for bot in bots:
    bloom_filter.insert(bot)

In [12]:
url = 'https://stream.wikimedia.org/v2/stream/recentchange'
wiki = 'enwiki' #Client side filter
counter = 0
changes = []
maxEvents =  1000 # print n events and stop
for event in EventSource(url):
    if event.event == 'message':
        try:
            change = json.loads(event.data)
        except ValueError:
            continue
        if change['wiki'] == wiki:
            counter += 1

            bot_bloom = bloom_filter.check(change['user'])
            change['bot_bloom'] = bot_bloom
            changes.append(change)

            if counter > maxEvents:
                break

In [ ]:
test = pd.json_normalize(changes)
test

In [18]:
test.loc[(test['bot']==True) & (test['bot_bloom']==False), ['bot', 'bot_bloom']]

,bot,bot_bloom
223,True,False


In [16]:
fp_rate = test[(test['bot']==False) & (test['bot_bloom']==True)].shape[0] / test.shape[0]
fn_rate = test[(test['bot']==True) & (test['bot_bloom']==False)].shape[0] / test.shape[0]
fp_rate, fn_rate

(0.14985014985014986, 0.000999000999000999)